In [2]:
import astropy.io.fits as fits
import numpy as np
import h5py as h5
path_data_cats = '/global/cfs/cdirs/m5099/RR2/Euclid_cats.npy'
cats_Euclid  = np.load(path_data_cats,allow_pickle=True).item()

In [3]:
catalog = h5.File('/pscratch/sd/d/dhayaa/For_Marco/DECADE/ShearCatalogDR3.hdf5','r')
for xx in ['NGC','SGC']:
    cat = dict()
    bin_ = np.array(catalog[xx] ['BIN'])
    ra = np.array(catalog[xx] ['RA'])
    dec = np.array(catalog[xx] ['DEC'])
    e1 = np.array(catalog[xx] ['g1'])
    e2 = np.array(catalog[xx] ['g2'])
    w = np.array(catalog[xx] ['mcal_g_w'])
    for i in range(1,5):
        mask = bin_ == i 
        cat[i-1] = dict()
        cat[i-1]['ra'] = ra[mask]
        cat[i-1]['dec'] = dec[mask]
        cat[i-1]['e1'] = e1[mask]- np.sum(e1*w)/np.sum(w)
        cat[i-1]['e2'] = e2[mask]- np.sum(e2*w)/np.sum(w)
        cat[i-1]['w'] = w[mask]
    np.save('/global/cfs/cdirs/m5099/DESY3/DECADE_{0}.npy'.format(xx),cat)
    

In [12]:
nz_SGC = np.load('/pscratch/sd/d/dhayaa/For_Marco/DECADE/nofz/SGC/mean_nz_Final.npy',allow_pickle=True)
nz_NGC = np.load('/pscratch/sd/d/dhayaa/For_Marco/DECADE/nofz/NGC/mean_nz_Final.npy',allow_pickle=True)


min_z   = 0.01
max_z   = 5
delta_z = 0.05
zbins   = np.arange(min_z,max_z+delta_z,delta_z)
zbinsc  = zbins[:-1]+(zbins[1]-zbins[0])/2. 

nz_RR2= dict()
nz_RR2['z_rebinned'] = zbinsc
nz_RR2['nz_rebinned'] = nz_NGC

np.save('/global/cfs/cdirs/m5099/DESY3/nz_NGC.npy',nz_RR2)


nz_RR2= dict()
nz_RR2['z_rebinned'] = zbinsc
nz_RR2['nz_rebinned'] = nz_SGC

np.save('/global/cfs/cdirs/m5099/DESY3/nz_SGC.npy',nz_RR2)


# DES Y6

In [35]:
import h5py as h5


In [38]:
import h5py as h5
import numpy as np
import healsparse
met = h5.File('/global/cfs/cdirs/des/y6kp-cats/final_version/metadetect_2024-11-07.hdf5','r')
# add ra,dec  ----------------
cat_met = dict()
# Loop over bins to collect data
for i in range(4):
    cat_met[i] = dict()


    # Collect weights
    ra_mdet = np.array(met['noshear'][f'tomo_bin_{i}']['ra']) 
    dec_mdet = np.array(met['noshear'][f'tomo_bin_{i}']['dec']) 

    cat_met[i]['ra'] = ra_mdet
    cat_met[i]['dec'] = dec_mdet

    
    ww = np.array(met['noshear'][f'tomo_bin_{i}']['w'])
    ww_1p = np.array(met['1p'][f'tomo_bin_{i}']['w'])
    ww_1m = np.array(met['1m'][f'tomo_bin_{i}']['w'])
    ww_2p = np.array(met['2p'][f'tomo_bin_{i}']['w'])
    ww_2m = np.array(met['2m'][f'tomo_bin_{i}']['w'])

    # Collect g1 and g2 values
    g1_noshear = (np.array(met['noshear'][f'tomo_bin_{i}']['gauss_g_1']))
    g2_noshear = (np.array(met['noshear'][f'tomo_bin_{i}']['gauss_g_2']))

    g1_1p = (np.array(met['1p'][f'tomo_bin_{i}']['gauss_g_1']))
    g1_1m = (np.array(met['1m'][f'tomo_bin_{i}']['gauss_g_1']))
    g1_2p = (np.array(met['2p'][f'tomo_bin_{i}']['gauss_g_1']))
    g1_2m = (np.array(met['2m'][f'tomo_bin_{i}']['gauss_g_1']))
    g2_1p = (np.array(met['1p'][f'tomo_bin_{i}']['gauss_g_2']))
    g2_1m = (np.array(met['1m'][f'tomo_bin_{i}']['gauss_g_2']))
    g2_2p = (np.array(met['2p'][f'tomo_bin_{i}']['gauss_g_2']))
    g2_2m = (np.array(met['2m'][f'tomo_bin_{i}']['gauss_g_2']))


    # Compute mean values and response matrices
    mean_g1 = np.sum(ww * g1_noshear) / np.sum(ww)
    mean_g1p = np.sum(ww_1p * g1_1p) / np.sum(ww_1p)
    mean_g1m = np.sum(ww_1m * g1_1m) / np.sum(ww_1m)
    R11 = (mean_g1p - mean_g1m) / (2 * 0.01)
    
    mean_g2 = np.sum(ww * g2_noshear) / np.sum(ww)
    mean_g2p = np.sum(ww_2p * g2_2p) / np.sum(ww_2p)
    mean_g2m = np.sum(ww_2m * g2_2m) / np.sum(ww_2m)
    R22 = (mean_g2p - mean_g2m) / (2 * 0.01)

    g1 = g1_noshear/(0.5*(R11+R22))
    g2 = g2_noshear/(0.5*(R11+R22))
    cat_met[i]['e1'] = g1-np.mean(g1*ww)/np.mean(ww)
    cat_met[i]['e2'] = g2-np.mean(g2*ww)/np.mean(ww)
    cat_met[i]['w'] = ww

np.save('/global/cfs/cdirs/m5099/DESY3/DESY6.npy',cat_met)


In [27]:
import numpy as np
m = np.load('/global/cfs/cdirs/m5099/DESY3/20k_m_nz_realizations.npz')
nz_RR2= dict()
nz_RR2['z_rebinned'] = np.arange(0,3,0.01)
nz_RR2['nz_rebinned'] = np.mean(m['nzs'],axis=0)
nz_RR2['nz_rebinned'][:,-1] = 0.
np.save('/global/cfs/cdirs/m5099/DESY3/nz_DESY6.npy',nz_RR2)


In [8]:
np.mean(m['ms'],axis=1)
np.mean(m['nzs'],axis=0)

array([-0.00343755,  0.0064513 ,  0.01591432,  0.00162992])

In [21]:
idx_rel = np.random.randint(0,20000,1)[0]
m['ms'][idx_rel]
m['nzs'][idx_rel][:,-1]

IndexError: index 18266 is out of bounds for axis 0 with size 4

array([0.30127065, 0.19114114, 0.15163738, 0.11113555])

In [ ]:
def make_WL_sample(ngal_glass, zeff_glass, cosmo_bundle, sims_parameters, nside_maps, fields, cats_Euclid, SC_corrections =None,do_catalog = False, include_SC = True, compact_savings = False):
    if include_SC:
        corr_variance_array =  [  SC_corrections['corr_variance_fit'][tomo](sims_parameters['bias_sc'][tomo])        for tomo in range(len(ngal_glass))]
        coeff_kurtosis_array = [  SC_corrections['coeff_kurtosis_fit'][tomo](sims_parameters['bias_sc'][tomo])       for tomo in range(len(ngal_glass))]
        A_corr_array = [  SC_corrections['A_corr_fit'][tomo](sims_parameters['bias_sc'][tomo])                       for tomo in range(len(ngal_glass))]
    else:
        corr_variance_array = np.ones(len(ngal_glass))
        coeff_kurtosis_array = np.zeros(len(ngal_glass))
        A_corr_array  = np.ones(len(ngal_glass))
        sims_parameters['bias_sc'] = np.zeros(len(ngal_glass))
    
    
    
    
    kappa_tot  = np.zeros((len(ngal_glass), 12*nside_maps**2))
    g1_tot     = np.zeros((len(ngal_glass), 12*nside_maps**2))
    g2_tot     = np.zeros((len(ngal_glass), 12*nside_maps**2))
    d_tot      = np.zeros((len(ngal_glass), 12*nside_maps**2))
    
    
    # load each lightcone output in turn and add it to the simulation
    # note: I added a -sign to gamma to match data conventions later
    for tomo in range(len(ngal_glass)):
        for i in (range(len(fields['gamma']))):       
            C1 = 5e-14
            rho_crit0_h2 = ccl.physical_constants.RHO_CRITICAL
            rho_c1 = C1 * rho_crit0_h2
            IA_f = F_nla(z=zeff_glass[i],
             om0=sims_parameters['Omega_m'],
             A_ia=sims_parameters['A_IA'], rho_c1=rho_c1, eta=sims_parameters['eta_IA'], z0=0.67,
             cosmo=cosmo_bundle['cosmo_pyccl'])
            
            g1_tot[tomo] += ngal_glass[tomo,i] * (-fields['gamma'][i][0].real-fields['IA_shear'][i][0].real*IA_f) * (1 + sims_parameters['bias_sc'][tomo] * fields['density'][i])
            g2_tot[tomo] += ngal_glass[tomo,i] * (-fields['gamma'][i][0].imag-fields['IA_shear'][i][0].imag*IA_f) * (1 + sims_parameters['bias_sc'][tomo] * fields['density'][i])
            d_tot[tomo]  += ngal_glass[tomo,i] * (1 + sims_parameters['bias_sc'][tomo] * fields['density'][i] )
       
    
    sims_parameters.setdefault('rot', 0)
    sims_parameters.setdefault('delta_rot', 0)

    maps_sim = dict()
    for tomo in range(len(ngal_glass)):
        maps_sim[tomo] = dict()



        pix_ = convert_to_pix_coord(cats_Euclid[tomo]['ra'],cats_Euclid[tomo]['dec'], nside=nside_maps*2)
        pix = rotate_and_rebin(pix_, nside_maps, sims_parameters['rot'], delta_=sims_parameters['delta_rot'])
             

        # source clustering term ~
        f = 1./np.sqrt(d_tot[tomo])
        f = f[pix]
    
    
        n_map = np.zeros(hp.nside2npix(nside_maps))
        n_map_sc = np.zeros(hp.nside2npix(nside_maps))
    
                        
        unique_pix, idx, idx_rep = np.unique(pix, return_index=True, return_inverse=True)
    
    
        n_map_sc[unique_pix] += np.bincount(idx_rep, weights=cats_Euclid[tomo]['w']/f**2)
        n_map[unique_pix] += np.bincount(idx_rep, weights=cats_Euclid[tomo]['w'])
    
        g1_ = g1_tot[tomo][pix]
        g2_ = g2_tot[tomo][pix]
    
    
        es1,es2 = apply_random_rotation(cats_Euclid[tomo]['e1']/f, cats_Euclid[tomo]['e2']/f)
        es1_ref,es2_ref = apply_random_rotation(cats_Euclid[tomo]['e1'], cats_Euclid[tomo]['e2'])
        es1a,es2a = apply_random_rotation(cats_Euclid[tomo]['e1']/f, cats_Euclid[tomo]['e2']/f)
    
    
        #x1_sc,x2_sc = addSourceEllipticity({'shear1':g1_,'shear2':g2_},{'e1':es1,'e2':es2},es_colnames=("e1","e2"))
    
    
        e1r_map = np.zeros(hp.nside2npix (nside_maps))
        e2r_map = np.zeros(hp.nside2npix (nside_maps))
        e1r_map0 = np.zeros(hp.nside2npix(nside_maps))
        e2r_map0 = np.zeros(hp.nside2npix(nside_maps))
        e1r_map0_ref = np.zeros(hp.nside2npix(nside_maps))
        e2r_map0_ref = np.zeros(hp.nside2npix(nside_maps))
        g1_map = np.zeros(hp.nside2npix(nside_maps))
        g2_map = np.zeros(hp.nside2npix(nside_maps))
    
        unique_pix, idx, idx_rep = np.unique(pix, return_index=True, return_inverse=True)
    
    
        e1r_map[unique_pix] += np.bincount(idx_rep, weights=es1*cats_Euclid[tomo]['w'])
        e2r_map[unique_pix] += np.bincount(idx_rep, weights=es2*cats_Euclid[tomo]['w'])
    
        e1r_map0[unique_pix] += np.bincount(idx_rep, weights=es1a*cats_Euclid[tomo]['w'])
        e2r_map0[unique_pix] += np.bincount(idx_rep, weights=es2a*cats_Euclid[tomo]['w'])
    
        e1r_map0_ref[unique_pix] += np.bincount(idx_rep, weights=es1_ref*cats_Euclid[tomo]['w'])
        e2r_map0_ref[unique_pix] += np.bincount(idx_rep, weights=es2_ref*cats_Euclid[tomo]['w'])
    
    
        mask_sims = n_map_sc != 0.
        e1r_map[mask_sims]  = e1r_map[mask_sims]/(n_map_sc[mask_sims])
        e2r_map[mask_sims] =  e2r_map[mask_sims]/(n_map_sc[mask_sims])
        e1r_map0[mask_sims]  = e1r_map0[mask_sims]/(n_map_sc[mask_sims])
        e2r_map0[mask_sims] =  e2r_map0[mask_sims]/(n_map_sc[mask_sims])
        e1r_map0_ref[mask_sims]  = e1r_map0_ref[mask_sims]/(n_map[mask_sims])
        e2r_map0_ref[mask_sims] =  e2r_map0_ref[mask_sims]/(n_map[mask_sims])
    
    
    
        var_ =  e1r_map0_ref**2+e2r_map0_ref**2
    
    
        #'''
        e1r_map   *= 1/(np.sqrt(A_corr_array[tomo]*corr_variance_array[tomo])) * np.sqrt((1+coeff_kurtosis_array[tomo]*var_))
        e2r_map   *= 1/(np.sqrt(A_corr_array[tomo]*corr_variance_array[tomo])) * np.sqrt((1+coeff_kurtosis_array[tomo]*var_))
        e1r_map0  *= 1/(np.sqrt(A_corr_array[tomo]*corr_variance_array[tomo])) * np.sqrt((1+coeff_kurtosis_array[tomo]*var_))
        e2r_map0  *= 1/(np.sqrt(A_corr_array[tomo]*corr_variance_array[tomo])) * np.sqrt((1+coeff_kurtosis_array[tomo]*var_))
    
    
    
        
        
        #'''
        g1_map[unique_pix] += np.bincount(idx_rep, weights= g1_*cats_Euclid[tomo]['w'])
        g2_map[unique_pix] += np.bincount(idx_rep, weights= g2_*cats_Euclid[tomo]['w'])
    
    
    
        g1_map[mask_sims]  = g1_map[mask_sims]/(n_map_sc[mask_sims])
        g2_map[mask_sims] =  g2_map[mask_sims]/(n_map_sc[mask_sims])

        if compact_savings:
            e1_ = ((g1_map+e1r_map0))[mask_sims]
            e2_ = ((g2_map+e2r_map0))[mask_sims]
            g1_ = g1_map[mask_sims]
            g2_ = g2_map[mask_sims]
            e1n_ = ( e1r_map)[mask_sims]
            e2n_ = ( e2r_map)[mask_sims]
            idx_ = np.arange(len(mask_sims))[mask_sims]
        
            maps_sim[tomo] =     {'g1_map':g1_,'g2_map':g2_,'e1':e1_,'e2':e2_,'e1n':e1n_,'e2n':e2n_,
                                    'idx_':idx_}

            
        else:
            e1_ = ((g1_map+e1r_map0))#[mask_sims]
            e2_ = ((g2_map+e2r_map0))#[mask_sims]
            e1n_ = ( e1r_map)#[mask_sims]
            e2n_ = ( e2r_map)#[mask_sims]
           # idx_ = np.arange(len(mask_sims))[mask_sims]
        
            maps_sim[tomo] =     {'g1_map':g1_map,'g2_map':g2_map,'e1':e1_,'e2':e2_,'e1n':e1n_,'e2n':e2n_,
                                    'e1r_map0_ref':e1r_map0_ref,
                                    'e2r_map0_ref':e2r_map0_ref,
                                    'var_':var_}     
    
        if do_catalog:
    
            cats_sim = dict()
            # make a catalog ---------------------------------------------------------------------------------------------------------------------------------
            SC_per_pixel_correction_noise  = f**2/((np.sqrt(A_corr_array[tomo]*corr_variance_array[tomo])) * np.sqrt((1+coeff_kurtosis_array[tomo]*var_)))[pix]
            
            # the f**2 applied to g1,g2 is the normalisation missing in the g1_tot,g2_tot ---------------------------------------------------
            e1_SC = g1_*f**2+es1a*SC_per_pixel_correction_noise
            e2_SC = g2_*f**2+es2a*SC_per_pixel_correction_noise
            #e1_SC,e2_SC = addSourceEllipticity({'shear1':g1_,'shear2':g2_},{'e1':es1a*SC_per_pixel_correction_noise,'e2':es2a*SC_per_pixel_correction_noise},es_colnames=("e1","e2"))
            cats_sim[tomo] =  {'ra':cats_Euclid[tomo]['ra'],'dec':cats_Euclid[tomo]['dec'],'e1':e1_SC,'e2':e2_SC,'w':cats_Euclid[tomo]['w']}
        
        else:
            cats_sim = None
            
    return maps_sim, cats_sim